In [1]:
#---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8

In [2]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

# run_settings = Pygeodyn.print_input(options=False, info=False)


settings_SPIRE= {# Basic input settings
                 'satellite'      : {'input': 'spire83'},
                 'den_model'      : {'input': 'jb2008'},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_test_infrastruc'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': 'CD_2p3'},
               # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':2.300000},
                  'scaling_factor'        : {'input':False},
                  'cd_adjustment_boolean' : {'input':False },
                  'hours_between_cd_adj'  : {'input':6 },
               # Run
                  'step'           : {'input': 10.},
                  #
#                   'arc'            : {'input':['2018.313',]},
                  'arc'            : {'input':['2018.314',]},
#                   'epoch_start'    : {'input':["2018-11-09 00:00:00"]},
#                   'epoch_start'    : {'input':["2018-11-09 00:05:35"]},
#                   'epoch_stop'     : {'input':["2018-11-10 00:00:00"]},
                  'epoch_start'    : {'input':["2018-11-10 00:00:00"]},
                  'epoch_stop'     : {'input':["2018-11-11 00:00:00"]},
#                   'initial_conditions':{'input':[-1367775.262313638,
#                                                 -5125780.359399426,
#                                                 -4361445.789580496,
#                                                 6645.941518838111,
#                                                 1215.197770462808,
#                                                 -3507.908078628219]} ,

                   
                  'global_options': {'input':'pso_2018'},
               # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil',
                                                'Density',
                                                'DragFile',
                                                #'Residuals_summary',
                                               ]},
              #end dict
              }

In [3]:
sat = Pygeodyn(settings_SPIRE, use_file=False)
# sat = Pygeodyn(settings_icesat, use_file=False)

sat.run_arcs()
obj = sat.getData()

/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-05/
/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-06/
/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-07/
/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-08/
/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-09/
/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-10/
files_with_sat ['/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-05/2018-11-05T00-41-19_2018-11-05T01-36-55_083/leoOrb_2018-11-05T00-41-19Z.1058054.083.sp3', '/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-05/2018-11-05T02-52-52_2018-11-05T03-46-55_083/leoOrb_2018-11-05T02-52-52Z.1058087.083.sp3', '/data/SatDragModelValidation/data/inputs/sat_spire83/data_Spire/arcs/2018-11-05/2018-11-05T05-02-49_2018-11-05T05-56-55_083/leoOrb_2018-11-05T05-02-49Z.105

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
arc = settings_SPIRE['arc']['input'][0]

X_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X j2000']
Y_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y j2000']
Z_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z j2000']
#
Xdot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X_dot j2000']
Ydot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y_dot j2000']
Zdot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z_dot j2000']

date_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC']

In [ ]:
date_prop

In [ ]:
epoch_start = settings_SPIRE['epoch_start']['input'][0]


prms_arc={}
prms_arc['epoch_start'] = epoch_start
prms_arc['epoch_startDT'] = pd.to_datetime(prms_arc['epoch_start'],\
                                            format='%Y-%m-%d %H:%M:%S')

epoch_startDT = prms_arc['epoch_startDT']
file_statevector_ICs = "/data/SatDragModelValidation/data/inputs/"\
                        +"sat_spire83/setups/Spire83_initialconditions_Nov2018_v1.txt"

datetype = 'datetime_string'

date_in_file_flag= False
import linecache

### Only need to use accuracy to within 1 second (ignore the microseconds in the file)

if datetype == 'datetime_string':
    date_str = str(epoch_startDT)
elif datetype == 'YYMMDDHHMMSS':
    date_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')


with open(file_statevector_ICs, 'r') as f:
    ### Find the dates that have the same hour    
    if datetype == 'datetime_string':
        date_roundhour_str = str(epoch_startDT)[:10]
    elif datetype == 'YYMMDDHHMMSS':
        date_roundhour_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')

    ### Scan through IC file and append a list of dates within the same hour
    line_no_list = []
    line_list = []
    with open(file_statevector_ICs, 'r') as f:
        for line_no, line_text in enumerate(f):
            if date_roundhour_str in line_text:
                line_no_list.append(line_no)
    for i in np.arange(line_no_list[0]-10, line_no_list[-1]+10):
        line = linecache.getline(file_statevector_ICs,i)
        line_list.append(line)
    dates = []
    for i ,val in enumerate(line_list):
        if datetype == 'datetime_string':
            dates.append(pd.to_datetime(line_list[i][:19],format='%Y-%m-%d %H:%M:%S'))
        elif datetype == 'YYMMDDHHMMSS':
            dates.append(pd.to_datetime(line_list[i][:19],format='%y%m%d%H%M%S.%f'))
            
            
            
xyzline = pd.read_csv(file_statevector_ICs, 
            skiprows = line_no_list[0], 
            nrows=line_no_list[-1]- line_no_list[0],           
            sep = '\s+',
            dtype=object,
            names = [
                'DateYMD',
                'DateHMS',
                'X',
                'Y',
                'Z',
                'X_dot',
                'Y_dot',
                'Z_dot',
                    ],)

xyzline['Date'] =  pd.to_datetime(xyzline['DateYMD']+xyzline['DateHMS'], format='%Y-%m-%d%H:%M:%S')
del xyzline['DateYMD'], xyzline['DateHMS']







In [ ]:
line_no_list[0]

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                })


fig = make_subplots(rows=3, cols=1)



fig.add_trace(go.Scattergl(
            x=pd.to_datetime(xyzline['Date']),
            y=xyzline['X'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(xyzline['Date']),
            y=xyzline['Y'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(xyzline['Date']),
            y=xyzline['Z'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

### ==============================================================


fig.add_trace(go.Scattergl(
            x=pd.to_datetime(date_prop),
            y=X_prop.values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(date_prop),
            y=Y_prop.values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(date_prop),
            y=Z_prop.values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

fig.update_yaxes( title="X",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Y",  exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="Z",  exponentformat= 'power',row=3, col=1)


fig.update_layout(title='PCE (blue, Spire LeoOrb POD); Orbit Fit (red, data reduction with PCE)',
        autosize=True
#         width=800,height=900,
                )
fig.show(config=config)


In [ ]:
# # First run of a new satellite:
#     - generate some sort of initial conditions file
#     - 

# Load input data
Load the data from whatever source, and define the format.

In [ ]:
# import numpy  as np
# import pandas as pd
# from  datetime import datetime,timedelta
# import os

In [ ]:
# #### INPUTS ----------------------------------------------------
dir_input = '/data/SatDragModelValidation/data/inputs'
dir_spire_sourcetraj = dir_input+'/sat_spire83/data_Spire/arcs'
# file_IC = dir_input+'/sat_spire83/setups/statevector_ICs_eci_epochA.txt'
# spire_sat_num = 83
# days          = np.arange(1,32)
# #### INPUTS ----------------------------------------------------  


In [ ]:

###  Make the file if not exist
if os.path.exists(file_IC):
    print(f"Raw initial conditions file: {file_IC}")
    print('File exists and does not need to be re-written.')
    
    
    epoch_start =settings_SPIRE['epoch_start']['input'][0]
    prms_arc={}
    prms_arc['epoch_start'] = epoch_start
    prms_arc['epoch_startDT'] = pd.to_datetime(prms_arc['epoch_start'],\
                                           format='%Y-%m-%d %H:%M:%S')
    #inputs----------------------------------
    datetype = 'datetime_string'
    file_statevector_ICs=file_IC                                     
    epoch_startDT = prms_arc['epoch_startDT']
    #inputs----------------------------------
    date_in_file_flag= False
    import linecache
    #
    ### Only need to use accuracy to within 1 second (ignore the microseconds)
    if datetype == 'datetime_string':
        date_str = str(epoch_startDT)
    elif datetype == 'YYMMDDHHMMSS':
        date_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')
    #
    with open(file_statevector_ICs, 'r') as f:
        for line_no, line_text in enumerate(f):
            if date_str in line_text:
                date_in_file_flag= True
                print('    ','xyzline',line_no,line_text)
                break
    if date_in_file_flag == False:
        ### Find the dates that have the same hour    
        if datetype == 'datetime_string':
            date_roundhour_str = str(epoch_startDT)[:10]
        elif datetype == 'YYMMDDHHMMSS':
            date_roundhour_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')
        ### Scan through IC file and append a list of dates within the same hour
        line_no_list = []
        line_list = []
        with open(file_statevector_ICs, 'r') as f:
            for line_no, line_text in enumerate(f):
                if date_roundhour_str in line_text:
                    line_no_list.append(line_no)
        for i in np.arange(line_no_list[0]-10, line_no_list[-1]+10):
            line = linecache.getline(file_statevector_ICs,i)
            line_list.append(line)
        dates = []
        for i ,val in enumerate(line_list):
            if datetype == 'datetime_string':
                dates.append(pd.to_datetime(line_list[i][:19],format='%Y-%m-%d %H:%M:%S'))
            elif datetype == 'YYMMDDHHMMSS':
                dates.append(pd.to_datetime(line_list[i][:19],format='%y%m%d%H%M%S.%f'))
    #
    xyzline = pd.read_csv(file_statevector_ICs, 
                skiprows = line_no_list[0], 
                nrows=line_no_list[-1]- line_no_list[0],           
                sep = '\s+',
                dtype=object,
                names = [
                    'DateYMD',
                    'DateHMS',
                    'X',
                    'Y',
                    'Z',
                    'X_dot',
                    'Y_dot',
                    'Z_dot',
                        ],)
    #
    xyzline['Date'] =  pd.to_datetime(xyzline['DateYMD']+xyzline['DateHMS'], format='%Y-%m-%d%H:%M:%S')
    del xyzline['DateYMD'], xyzline['DateHMS']
#     pass
else:
    
    #### Find files throught dir tree containing Satellite's LeoOrb POD
    files_with_sat = []
    ## Loop through the daily directories
    for iday,day in enumerate(days):
        dir_set   = dir_spire_sourcetraj + f'/2018-11-{day:02d}/'
        #
        ## Loop through the arc directories for that day,
        ## Identify the leoOrb..._{sat#}.sp3 (Spire "POD") files
        for root, dirs, files in os.walk(dir_set, topdown=False):
            for name in files:
                if 'leoOrb'in name and f'{spire_sat_num:03d}.sp3' in name:
                    files_with_sat.append(os.path.join(root, name))
    del root, dirs, files, name
    del days, iday, day



    #### Load raw data from all found files
    from pygeodyn.satellite_spire import read_SpireLeoOrbPOD_sp3c
    leoOrb_ecef = read_SpireLeoOrbPOD_sp3c(files_with_sat)
    del files_with_sat
    gc.collect()



    ## Load coordinate transformation functions
    from pygeodyn.util_dir.coordinate_systems import iau06_Call_ecef2eci
    from pygeodyn.util_dir.time_systems import time_gps_to_utc,\
                                               get_leapseconds, jday           
    ### Get the leapseconds for time period of interest
    dAT = get_leapseconds(leoOrb_ecef['date_gps'][0].year,
                          leoOrb_ecef['date_gps'][0].month,
                          leoOrb_ecef['date_gps'][0].day,)
    ## Convert to UTC
    leoOrb_ecef['date_utc'] = [time_gps_to_utc(time, dAT) \
                                  for time in leoOrb_ecef['date_gps']]
    del leoOrb_ecef['date_gps']
    del leoOrb_ecef['clock_microsec']
    gc.collect()



    ### Convert all ephemerides from ECEF-igs08 to ECI-j2000
    import time
    start = time.time()
    conv_dms2kms = 1/10000.
    #
    len_ecef = np.shape(leoOrb_ecef['date_utc'])[0]    
    
    
    
    f = open(file_IC, "w")
    f.write("\n")
    f.close()

    ### Write to file
    with open(file_IC, 'r+') as file:

        #### Manually write the header units
        header_units =\
                 f"{'UTC'.rjust(len(str(leoOrb_ecef['date_utc'][1]))-1,' ') }"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\

        #### Manually write the header field names
        header_names =\
                 f"{'Date'.rjust(len(str(leoOrb_ecef['date_utc'][1]))-1,' ') }"\
                +f"  {'X'.rjust(15,' ')}"\
                +f"  {'Y'.rjust(15,' ')}"\
                +f"  {'Z'.rjust(15,' ')}"\
                +f"  {'X_dot'.rjust(15,' ')}"\
                +f"  {'Y_dot'.rjust(15,' ')}"\
                +f"  {'Z_dot'.rjust(15,' ')}"\

        #### Manually write the detailed header description
        header_meta = \
        f'''### Initial conditions file
    ### -----------------------
    ###     Satellite: Spire_083 (1804607)
    ###     Epoch: +start____ 2018  9 23  0 12  0.0300000 
    ###            +stop_____ 2018 12  9 23 43 35.9080000 
    ###     Last modified: {datetime.now()-timedelta(hours=7)}
    ###
    ### Source
    ### -------
    ###     leoAtt_2018-09-23T00-12-00Z.9999999.083.log
    ###     (long quaternion file)
    ###     %spire version 1.3  revision   1 2019 07 05 00:00 Spire     Spire Processing Center
    ###
    ### Contents
    ### --------
    ###     Date: (YYYY-MM-DD hh:mm:ss.ssssss) (UTC, converted from gps time)
    ###     pvi: Position and velocity (X, Y, Z, X_dot, Y_dot, Z_dot)
    ###          coordinate: ECI-J2000, (converted from ECEF-IGS08)
    ###          unit: m
    ###
    #{header_units}
    #{header_names}
    ### %eoh
    '''
        
        file.write(header_meta)  
        for indx,valdate in enumerate(leoOrb_ecef['date_utc']):
            ## position vector earth fixed  ( km )
            recef = [leoOrb_ecef['x_km'][indx],  
                     leoOrb_ecef['y_km'][indx], 
                     leoOrb_ecef['z_km'][indx]]
            ## velocity vector earth fixed  (km/s)
            vecef = [leoOrb_ecef['xdot_dms'][indx]*conv_dms2kms, 
                     leoOrb_ecef['ydot_dms'][indx]*conv_dms2kms,
                     leoOrb_ecef['zdot_dms'][indx]*conv_dms2kms]
            ### convert the data to correct coordinate ref frame
            (reci, veci, _)= iau06_Call_ecef2eci(\
                                          recef, vecef, None,
                                          leoOrb_ecef['date_utc'][indx].year  , 
                                          leoOrb_ecef['date_utc'][indx].month , 
                                          leoOrb_ecef['date_utc'][indx].day   , 
                                          leoOrb_ecef['date_utc'][indx].hour  , 
                                          leoOrb_ecef['date_utc'][indx].minute, 
                                          leoOrb_ecef['date_utc'][indx].second, 
                                          calc_accel = False)
        #### Manually write each row of the data.
            row =   f"{leoOrb_ecef['date_utc'][indx]}"\
                   +f"  {reci[0,0]*1000:15.5f}"\
                   +f"  {reci[1,0]*1000:15.5f}"\
                   +f"  {reci[2,0]*1000:15.5f}"\
                   +f"  {veci[0,0]*1000:15.5f}"\
                   +f"  {veci[1,0]*1000:15.5f}"\
                   +f"  {veci[2,0]*1000:15.5f}"\
                   +f"\n"
            file.write(row)
    #
    end = time.time()
    elapsed = end - start
    #
    print()
    print(f'       indxes: 0 -',indx,'') 
    print(f'       Time: ',np.round(elapsed,5),'secs') 


In [ ]:
# xyzline

# Making PCE:
 1. Identify source ephemeris   
    - format and define  
  
  
 2. Prepare ephemeris for fortran code  
     - convert to asciitraj.txt file
 
 3.   

## Make ascii txt file

In [ ]:
from pygeodyn.util_dir.time_systems import  time_utc_to_gps,\
                                            ymdhms_to_mjds


In [ ]:
# ### Prepare inputs
# import math

# pce_in = {}

# ### Convert UTC to GPS time and convert to MJDsec
# mjdsec_gps = [ymdhms_to_mjds(time_utc_to_gps(date, 37).year,
#                                        time_utc_to_gps(date, 37).month,
#                                        time_utc_to_gps(date, 37).day,
#                                        time_utc_to_gps(date, 37).hour,
#                                        time_utc_to_gps(date, 37).minute,
#                                        time_utc_to_gps(date, 37).second)
#                                            for date in xyzline['Date'].values]


# pce_in['mjdsec_gps'],\
# pce_in['frac_sec']    = map(list,  \
#                             zip(*[[int(math.modf(date)[1]),\
#                                    math.modf(date)[0]]\
#                                   for date in mjdsec_gps ]))
# pce_in['gps_offset'] = -18.0
# pce_in['X_j2000_m']          = xyzline['X'].values.astype(float)
# pce_in['Y_j2000_m']          = xyzline['Y'].values.astype(float)
# pce_in['Z_j2000_m']          = xyzline['Z'].values.astype(float)



In [ ]:
# import pandas as pd

# pce_in_df = pd.DataFrame(pce_in)
# pce_in_df.insert(1, 'utc_offset' , pce_in_df['frac_sec'] + pce_in_df['gps_offset'])

# del pce_in_df['frac_sec']
# del pce_in_df['gps_offset']
# # pce_in_df


# ##### Save as an unadorned txt file
# pce_in_df.to_csv('TRAJ.txt',      \
#                       sep=' ',      \
#                       index = False,\
#                       header=False)


## call fortran function

- pass paths into fortran using env vars
- fix the input options to be satellite specific
- add checks and writes to the fortran function

- re-write the function so that it is more clear.

In [ ]:
# import subprocess



# path_to_data = '/data/SatDragModelValidation/notebooks/'
# path_to_PCE_fortran = '/data/SatDragModelValidation/pygeodyn/pygeodyn/util_preprocessing/'
# path_preprocessing  = '/data/SatDragModelValidation/pygeodyn/pygeodyn/util_preprocessing/'
# in_filename  = 'TRAJ.txt'
# out_filename = 'g2b_pce'

# ### CHANGE DIRECTORY to where the fortran code is hosted
# os.chdir(path_to_PCE_fortran)

# #### Write the inputs to the fortran code to a file to be read in variably:
# file_FTN_opts = open(path_to_PCE_fortran +"options_fortrancode.txt","w+")
# file_FTN_opts.writelines(path_to_data+'/'+ '\n') 
# file_FTN_opts.writelines(in_filename     +'\n')
# file_FTN_opts.writelines(out_filename    +'\n')
# file_FTN_opts.close()        



# #### Compile the pce code:
# command_1 = './compile_pce_f.sh'
# subprocess.run(command_1, shell = True)
# print('pce_fortran.f compiled')

# #### Execute the pce code
# command_2 = './ExecutePCE.exe > out_pce 2> err_execute'
# subprocess.run(command_2, shell = True)
# print('pce_fortran.f executed')
# print('')

# # os.system('gzip -vr '+ path_to_data+'/'+out_filename)

# # os.system('mv '+ path_to_data+'/'+out_filename+'.gz'+ ' '+'/data/data_geodyn/inputs/icesat2/g2b/')
# # os.system('rm'+' '+path_preprocessing + '/TRAJ.txt')

# # if os.path.exists('/data/data_geodyn/inputs/icesat2/g2b/'+out_filename+'.gz'):
# #     print('The G2B file has been saved to: ','/data/data_geodyn/inputs/icesat2/g2b/',out_filename,'.gz')
# # else:
# #     print('The G2B binary file has been saved to: ',path_to_data,'/',out_filename, sep='')




In [ ]:
import copy
tabtab = '        '

In [ ]:
# """
#     TRAJ.txt for input to PCE_converter.f

# Requirements:
#     - File titled TRAJ.txt
#     - Each line of TRAJ.txt has one integer and four floating point words

#         Word #     Fmt     Description    
#         ------     ---     -----------
#            1       int     MJDSEC_secs_timeGPS
#            2      float    RSEC_fractional_secs + GPS_offset_secs_utc
#            3      float    X   (j2000)
#            4      float    Y   (j2000)
#            5      float    Z   (j2000)

# The integer and the first floating point word form the time tag of the 
#     record (this is just the first word of the RVG data record converted 
#     to an int).
#     The first floating point is the sum of words 2 and 3 of the RVG data 
#     record.
#     The last three words are the X, Y and Z coordinates of the satellite
#     in the J2000 coordinate system. The X, Y and Z values are words 
#     5, 6 and 7 of the RVG data record.
# """

In [ ]:


# #### First call the functions that convert RVG files to a dataframe


# print()
# print(tabtab,'=======================================================')
# print(tabtab,'STEP 3: Make ASCII.txt file to be read by FORTRAN code.')
# print(tabtab,'=======================================================')
# print()

# df_traj_txt = copy.deepcopy(RVG_FINAL)

# #### We need to prepare the dataframe to be saved to a CSV according 
# ####        to the requirements of the fortran code
# #### first integer of TRAJ.txt is just the first word 
# ####      of RVG data record converted to an integer 
# df_traj_txt.insert(0, 'first_int', df_traj_txt['MJDSEC_secs_timeGPS'].astype(int))

# del df_traj_txt['Date']
# del df_traj_txt['MJDSEC_secs_timeGPS']
# del df_traj_txt['XDOT_statevector_m_s']
# del df_traj_txt['YDOT_statevector_m_s']
# del df_traj_txt['ZDOT_statevector_m_s']

# #### The first floating point of TRAJ.txt is just 
# #        the sum of words 2 and 3 of the RVG data record
# sum_words_2_and_3 = df_traj_txt['RSEC_fractional_secs'] + df_traj_txt['GPS_offset_secs_utc']
# df_traj_txt.insert(1, 'first_float', sum_words_2_and_3)
# del df_traj_txt['RSEC_fractional_secs'] 
# del df_traj_txt['GPS_offset_secs_utc']

# ##### Save as a txt file with pandas built-in function
# df_traj_txt.to_csv(path_preprocessing + '/TRAJ.txt', sep=' ', index = False, header=False)

# del df_traj_txt
